# Bước 1: Tải Thư viện cần thiết

In [1]:
!pip install gensim
!pip install underthesea
!pip install tensorflow_hub
!pip install tensorflow

^C


You should consider upgrading via the 'C:\Users\ACER\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import torch
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
from underthesea import word_tokenize

In [ ]:
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf

import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# nltk.download('punkt')

# Bước 2: Tiền xử lý văn bản

In [ ]:
class TextProcessor:
    def __init__(self, df):
        self.df = df.copy()
        self.clean_text()
        self.elmo_model = hub.load("https://tfhub.dev/google/elmo/3")
        self.y = self.df["label"].astype(str)

    def clean_text(self):
        self.df["clean_text"] = self.df["text"].str.lower()
        self.df["clean_text"] = self.df["clean_text"].str.replace(r"[{}]".format(string.punctuation), "", regex=True)
        self.df["clean_text"] = self.df["clean_text"].str.replace(r"\d+", "", regex=True)
        # self.df["clean_text"] = self.df["clean_text"].apply(lambda x: " ".join(word_tokenize(x)))

    def tfidf_vectorizer(self, max_features=100):
        vectorizer = TfidfVectorizer(max_features=max_features)
        X_tfidf = vectorizer.fit_transform(self.df["clean_text"])
        return X_tfidf, self.y

    def bag_of_words_vectorizer(self, max_features=100):
        vectorizer = CountVectorizer(max_features=max_features)
        X_bow = vectorizer.fit_transform(self.df["clean_text"])
        return X_bow, self.y

    def one_hot_encoding(self):
        encoder = OneHotEncoder(sparse_output=False)
        one_hot_encoded = encoder.fit_transform(self.df[["clean_text"]])
        return one_hot_encoded, self.y

    def elmo_embeddings(self):
        def elmo_vectors(texts):
            return self.elmo_model.signatures["default"](tf.convert_to_tensor(texts))["elmo"]

        elmo_embeddings = elmo_vectors(self.df["clean_text"].tolist())
        return elmo_embeddings.numpy(), self.y

In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv("generated_soccer_questions.csv")

# Chuyển thành DataFrame
# df = pd.DataFrame(corpus, columns=["text", "label"])
df = df.sample(frac=0.5, random_state=42)  # Lấy 50% dữ liệu
df.reset_index(drop=True, inplace=True)

# Hiển thị một số dòng của tập dữ liệu
print("Dữ liệu mẫu:")
print(df.head())

In [ ]:
# Tiền xử lý văn bản
processor = TextProcessor(df)

X_tfidf, y_tfidf = processor.tfidf_vectorizer()
X_bow, y_bow = processor.bag_of_words_vectorizer()
X_one_hot, y_one_hot = processor.one_hot_encoding()
X_elmo, y_elmo = processor.elmo_embeddings()

X_train, X_test, y_train, y_test = train_test_split(X_bow, y_bow, test_size=0.3, random_state=42, stratify=y_bow)

print("TF-IDF Shape:", X_tfidf.shape)
print("BoW Shape:", X_bow.shape)
print("One-Hot Shape:", X_one_hot.shape)
print("ELMo Shape:", X_elmo.shape)

# Bước 3: Vaive Bayes

In [ ]:
# Huấn luyện Naive Bayes với Laplace smoothing
nb_model = MultinomialNB(alpha=0.1)
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
nb_acc = accuracy_score(y_test, y_pred_nb)
print("Naive Bayes Accuracy:", nb_acc)
print(classification_report(y_test, y_pred_nb))

# Bước 4: Logistic Regression

In [ ]:
# Huấn luyện Logistic Regression với regularization cao hơn
lr_model = LogisticRegression(max_iter=1000, C=0.1)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, y_pred_lr)
print("Logistic Regression Accuracy:", lr_acc)
print(classification_report(y_test, y_pred_lr))

# Bước 5: Decision Tree

In [ ]:
# Huấn luyện Decision Tree với độ phức tạp thấp hơn
dt_model = DecisionTreeClassifier(max_depth=5, min_samples_split=20, random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
dt_acc = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", dt_acc)
print(classification_report(y_test, y_pred_dt))


# Bước 6: Doc2Vec

In [ ]:
# Biểu diễn bằng Doc2Vec
tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(df["clean_text"])]
d2v_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=2, workers=4, epochs=30)
X_d2v = np.array([d2v_model.infer_vector(text.split()) for text in df["clean_text"]])
X_train_d2v, X_test_d2v, y_train_d2v, y_test_d2v = train_test_split(X_d2v, y_bow, test_size=0.3, random_state=42, stratify=y_bow)

In [ ]:
# Huấn luyện với Doc2Vec
lr_d2v = LogisticRegression(max_iter=1000, C=0.5)
lr_d2v.fit(X_train_d2v, y_train_d2v)
y_pred_d2v = lr_d2v.predict(X_test_d2v)
d2v_acc = accuracy_score(y_test_d2v, y_pred_d2v)
print("Logistic Regression (Doc2Vec) Accuracy:", d2v_acc)
print(classification_report(y_test_d2v, y_pred_d2v))

# Bước 7: So sánh

In [ ]:
# So sánh mô hình
models = ["Naive Bayes", "Logistic Regression", "Decision Tree", "Logistic Regression (Doc2Vec)"]
accuracies = [nb_acc, lr_acc, dt_acc, d2v_acc]

plt.figure(figsize=(8,5))
sns.barplot(x=models, y=accuracies, palette="coolwarm")
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.xlabel("Model")
plt.title("So sánh độ chính xác của các mô hình")
plt.xticks(rotation=15)
plt.show()